# 6.804 Final Project

Meia Alsup, Sravya Bhamidipati, Anelise Newman

In [1]:
from string import digits
import json
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image, display
from IPython.core.display import HTML 
import urllib.request as request

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import nltk

In [2]:
 #ocr = pkl.load(open('raw_ocr_output.pickle','rb'))
#less_ocr = {key: ocr[key] for i, key in enumerate(ocr) if i < 100}
#less_ocr

In [4]:
#f = open('icon_annotations_testing.pickle','rb')
#icons = pkl.load(f)
#f.close()

## Clean input to just English words



In [10]:
f = open('google_text_extraction_output.pckl','rb')
infographic_name_to_words = pkl.load(f)
f.close()

print('We have extracted words from %d infographics.'%len(infographic_name_to_words))

We have extracted words from 62361 infographics.


In [11]:
def contains_english(set_of_words):
    english = set(['the', 'she', 'different', 'time', 'and', 'client', 'sports', 'grammar', 'family', 'report'])
    spanish = set(['el', 'es', 'cuando', 'esta', 'malo', 'que', 'tiene', 'para'])
    if bool(set(set_of_words) & spanish):
        return False
    if bool(set(set_of_words) & english):
        return True
    return False

In [12]:
english  = {key: val for key, val in infographic_name_to_words.items() if contains_english(val)}
len(english)

46952

In [11]:
json.dump(english, open('filtered.json', 'w'))
json.dump([key for key, val in english.items()], open('filtered_keys.json', 'w'))

In [19]:
words = [list_words for list_words in english.values()]
newList = []
stemmer = PorterStemmer()
for wordList in words:
    thisList = []
    for word in wordList:
        if len(word) > 2 and word.isalpha() and word not in STOPWORDS and wordnet.synsets(word):
            thisList.append(word.lower())
            #thisList.append(stemmer.stem(WordNetLemmatizer().lemmatize(word.lower(), pos='v')))
    newList.append(thisList)

In [39]:
NUM_TOPICS = 20

In [40]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(newList)
common_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

common_corpus = [common_dictionary.doc2bow(text) for text in newList]
# Train the model on the corpus.
id2token = {val:key for key, val in common_dictionary.token2id.items()}

In [41]:
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=NUM_TOPICS)

In [42]:
for i in range(NUM_TOPICS):
    terms = lda.get_topic_terms(i, 20)
    print(f'Topic number {i}')
    val = {id2token[t[0]]: t[1] for t in terms}
    #print(f'Top 20 terms: {val}')
    print(f'Top 20 terms: {val.keys()}')

Topic number 0
Top 20 terms: dict_keys(['home', 'price', 'market', 'real', 'average', 'property', 'value', 'estate', 'money', 'sales', 'house', 'prices', 'buy', 'new', 'stock', 'buying', 'sell', 'london', 'homes', 'investment'])
Topic number 1
Top 20 terms: dict_keys(['social', 'media', 'marketing', 'content', 'customer', 'customers', 'people', 'email', 'use', 'twitter', 'brand', 'business', 'post', 'marketers', 'service', 'page', 'blog', 'share', 'create', 'like'])
Topic number 2
Top 20 terms: dict_keys(['million', 'united', 'states', 'world', 'new', 'years', 'number', 'south', 'population', 'state', 'countries', 'north', 'year', 'people', 'national', 'total', 'average', 'america', 'american', 'age'])
Topic number 3
Top 20 terms: dict_keys(['users', 'online', 'internet', 'google', 'mobile', 'search', 'web', 'million', 'video', 'website', 'site', 'billion', 'social', 'use', 'user', 'sites', 'digital', 'content', 'data', 'people'])
Topic number 4
Top 20 terms: dict_keys(['energy', 'wate

In [43]:
saved = lda.save('saved_lda_15_topics')

## Intersction with infographics that have labeled bounding boxes

In [ ]:
# annotations from task where participants marked bounding boxes around all icons on an infographic
icon_dict = pkl.load(open( "icon_annotations_all.pickle", "rb") )
icons_english  = set(icon_dict.keys())

In [17]:
base='http://thumbnails.visually.netdna-cdn.com'

In [13]:
intersection = set([url_ for url_ in english.keys() if url_ in icons_english])

In [15]:
len(intersection)

1079

In [18]:
for url_ in intersection:
    #display(Image(url=f'{base}/{url_}'))
    request.urlretrieve(f'{base}/{url_}', f'images/{url_}')

## See category labels for chosen 